# Nurse Scheduling Problem

This notebook demonstrates how to use **qubo** and slack variables to model the nurse scheduling problem(NSP). The model will be analyzed and transformed by using **pyqubo** to matrix elements. 

In [1]:
import pandas as pd
import numpy as np
import requests
import calendar
import re
import json

from pyqubo import Binary, Array, Constraint, Model, And, Or, Not, Num

from lib.util import *

In [2]:
def convert_configuration_to_solution(config, variables):
    data = {}
    for var in variables:
        if config[str(variables.index(var))]:
            data[var] = 1
        else:
            data[var] = 0
    return data

def generate_shift_from_solution(per_grave, days, solution_dict):
    table = np.zeros((per_grave, days))
    for key, value in solution_dict.items():
        if "Graveyard" in key and "*" not in key:
            indexes = re.findall(r'\[(\d+)\]', key)
            indexes = [int(index) for index in indexes]
            table[indexes[0]][indexes[1]] = value
#     table = table.reshape(per_gravdays).astype(int)
    return table

def get_weekend_date(year, month):
    """
    Get the weekend date of a month
    :param year: int
    :param month: int
    :return: list
    """
    calendar_matrix = calendar.monthcalendar(year, month)
    weekend_date = []
    weekends = []
    # complexity: number of weekend_date
    for week in calendar_matrix:  # 4~5
        weekend_date = week[-2:]
        for date in weekend_date:  # 2
            if date > 0:
                weekends.append(date - 1)

    return weekends

def convert_hamiltonian_to_binary_polynomial_term(hamiltonian, variables):
    model = hamiltonian.compile()
    qubo, offset = model.to_qubo()
    # variables = model.variables
    binary_polynomial_terms = []

    for key, value in qubo.items():
        # print(key, value)

        binary_polynomial_terms.append({
            'c' : value,
            'p' : [
                variables.index(key[0]),
                variables.index(key[1])
            ]
        })

    binary_polynomial_terms.append({
        'c' : offset,
        'p' : []
    }) 

    return binary_polynomial_terms

In [3]:
def decode(solution, solution_index, model, number_of_workers, days):

    with open("solution{}.json".format(job_id), 'w') as file:
        json.dump(solution, file, indent=4)

    solution_dict = []
    for i in range(len(solution['qubo_solution']['solutions'])):
        solution_dict.append(
            convert_configuration_to_solution(solution['qubo_solution']['solutions'][i]['configuration'], model.variables))

    decoded_sample = model.decode_sample(solution_dict[solution_index], vartype='BINARY')
    print(json.dumps(solution['qubo_solution']['progress'], indent=4))
    print(json.dumps(decoded_sample.constraints(), indent=4))

    table = generate_shift_from_solution(number_of_workers, days, decoded_sample.sample)
    table = pd.DataFrame(table, dtype=int, columns=range(1, days+1))
    
    return table

## Constraint Functions

Here defines the constraint function class. The class should have the ability to calculate its hamiltonian and the evaluate of the constraint after having the result from the digital annealing solver.

Origianlly, the **pyqubo** and the **bqm** have function to know if the result satifies with the constraints and gives the programmer True and False. However, only True and False values are not sufficient to represent the quality of result. The result is not always perfect, and would have some constraints that are challenging to be satisfied, but the digital annealing solver has tried its best to lower down the energy. In view of this, the correctness ratio is necessary.

the abstract base class of constraint function should be in the following form:

In [4]:
class ConstraintFunction(object):
    
    def __init__(self, X:Array):
        self._X = X
        
    def hamiltonian(self) -> Model:
        pass
    
    def evaluate(self, table):
        pass

In the above snippet of code, the `hamiltonian` method should return the `Model` type of variable that represent the constraint defined in this class. For the `evaluate` method, it returns float numbers which are the quality  and std of the shift table for this constraint. The higher the value, the higher the quality and vice versa.

In [5]:
class ExpectedWorkingDays(ConstraintFunction):
    
    def __init__(self, X:Array, expected_working_days):
        super().__init__(X)
        
        self._expected_working_days = expected_working_days
        
    def hamiltonian(self):
        workers, days = self._X.shape
        H = Num(0)
        for r in range(workers):
            H += (sum([self._X[r][day] for day in range(1, days)], start=self._X[r][0]) - self._expected_working_days)**2
        return H
    
    def evaluate(self, table):
        
        content = table.values
        rows, cols = content.shape
        
        failed_rates = []
        for r in range(rows):
            diff = self._expected_working_days - sum(content[r])
            if (diff != 0):
                print("Worker %d failed => %d" % (r, diff))
            failed_rates.append(abs(diff) / self._expected_working_days)
         
        
        return np.average(failed_rates), np.std(failed_rates)

#### Expected Number of Workers in Each Shift

**Terminology**
*Shift*: A day in the shift table

The following constraint defines the expected number of workers in each shift. This constraint is common and used to balance the workforce and the workload on each day. The constraint is modeled by summing up the variables on each shift and minus the expected number of workers and, finally, double themselves and make them quadratic.

As for the `evaluate` method, it scans the shift day for each day and check if the shifts satisfy the constraint or not. The method would return a float number that represent the rate of correction. In addition, the method prints out which date is failed.

In [6]:
class ExpectedNumberOfWorkersInEachShift(ConstraintFunction):
    
    def __init__(self, X:Array, expected_workers):
        
        super().__init__(X)
        self._expected_workers = expected_workers
        
    def hamiltonian(self):
        shifts = []
        workers, days = X.shape
        H = 0
        for i in range(days):
            shift = 0
            for j in range(workers):
                shift += X[j][i]
            H += (shift - self._expected_workers) ** 2
            
        return H
    
    def evaluate(self, table):
        print("Evaluate on expected number of workers on each shift")
        failed = 0
        for i in table.columns:
            _sum = table[i].sum()
            if _sum < self._expected_workers:
                failed += 1
                print("\tdate[%d] Failed" % (i))
        correct_rate = failed / len(table.columns)
        
        return 1 - correct_rate

#### Max Consecutive Shifts

The constraint setup a limit on maximum number of consecutive shifts. The following constraint uses the slack variables to model the constraint. However, the use of slack variables increase the scale of the problem, i.e. the number of variables, and make the problem intractable. In view of this, the Fujitsu Limited **inequalities** terms to decrease the scale of the problem.

In [7]:
class MaxConsecutiveShifts(ConstraintFunction):
    
    def __init__(self, X:Array, max_consecutive_day):
        super().__init__(X)
        
        self._max_consecutive_day = max_consecutive_day
        
    
    def hamiltonian(self):
        shift_cycles = []
        row, col = self._X.shape
        for i in range(row):
            shift_cycle = []
            for j in range(col - self._max_consecutive_day):
                shift_cycle.append( sum([self._X[i][p] for p in range(j, j + self._max_consecutive_day)], start=self._X[i][j+self._max_consecutive_day]) )
            shift_cycles.append(shift_cycle)
        
        cycle = col - self._max_consecutive_day
        
        slack_initial = Array.create("slack1", shape=row * cycle * self._max_consecutive_day, vartype="BINARY")
        slack = np.zeros(row * cycle * self._max_consecutive_day).reshape(row, cycle * self._max_consecutive_day)
        slack = slack.tolist()
        print(slack_initial.shape)

        for i in range(row):
            for j in range(cycle * self._max_consecutive_day):
                slack[i][j] = slack_initial[cycle * self._max_consecutive_day * i + j]
        
        H = 0
        for i in range(row):
            for j in range(cycle):
                # To adapt the condition k is not equal to 4, a for-loop needed
                H += (shift_cycles[i][j] - sum(slack[i][l]
                                                for l in range(self._max_consecutive_day * j, self._max_consecutive_day *(j+1))) )**2
                
        return H
    
    def evaluate(self, table):
        print("Evaluate on the limit of max consecutive shifts:")
        content = table.values
        row, col = content.shape
        failed = 0
        for r in range(row):
            for c in range(col - self._max_consecutive_day - 1):
                if(sum(content[r][c:c+self._max_consecutive_day+1]) > self._max_consecutive_day):
                    failed += 1
                    print("\tworker[%d], date[%d] Failed" % (r+1, c+1))

        correctness_rate = 1 - (failed / (row*(col - self._max_consecutive_day)))
        
        return correctness_rate

#### Max Consecutive Shifts Inequalities

This constraint inherit the above constraint, sharing the same `evaluate` method and override the hamiltonian method. Please refer to the [link](https://portal.aispf.global.fujitsu.com/apidoc/da/jp/api-ref/da-qubo-v3-en.html) to make yourself familiar with the way to model the inequalities on DAU service. This can decrease the number of the variables and make the annealing progress focus on the critical variables instead of stack variables and intermediate terms.

In [8]:
class MaxConsecutiveShiftsInequalities(MaxConsecutiveShifts):
    
    def __init__(self, X, max_consecutive_days):
        super().__init__(X, max_consecutive_days)
    
    def hamiltonian(self):
        
        return Num(0)
    
    
    def inequalities(self, variables: list):
        shift_cycles = []
        row, col = self._X.shape
        for i in range(row):
            shift_cycle = []
            for j in range(col - self._max_consecutive_day):
                shift_cycle.append(sum([self._X[i][p] for p in range(j, j+self._max_consecutive_day+1)], start=Num(-self._max_consecutive_day)))
            shift_cycles.append(shift_cycle) 
        terms = []
        for shift_cycle in shift_cycles:
            for cycle in shift_cycle:
                term = convert_hamiltonian_to_binary_polynomial_term(cycle, variables)
                terms.append(term)
            
        return terms

#### Two Consecutive Shift

This constraint is placed to make each worker has at least two consecutive shifts.

In [9]:
class TwoConsecutiveShift(ConstraintFunction):
    
    def __init__(self, X:Array):
        super().__init__(X)
        
    def hamiltonian(self):
        row, col = self._X.shape
        Hc = 0
        for i in range(row):
            for j in range(col - 2):
                Hc = Hc + And(self._X[i][j + 1], 1 - Or(self._X[i][j], self._X[i][j + 2]))
            Hc = Hc + (And(self._X[i][0], Not(self._X[i][1]))) + (And(self._X[i][col - 1], Not(self._X[i][col - 2])))
        return Hc
        
    def evaluate(self, table):
        
        content = table.values
        row, col = content.shape
        failed = 0
        for r in range(row):
            for c in range(1, col-1):
                if content[r][c] == 1 and content[r][c-1] == 0 and content[r][c+1] == 0:
                    failed += 1
            if (content[r][0] == 1 and content[r][1] == 0) or (content[r][col - 1] == 1 and content[r][col - 2] == 0):
                failed += 1
                    
        possible_outcomes = row * col
        
        return 1 - (failed / possible_outcomes)

### Day-off constraint

There are two predominant constraints on day-off. One focus on the days off in a range of days, and the other is on the relationship between each day-off

#### WeekdayLeaveConstraint

The first is `WeekdayLeaveConstraint` which is imposed to enable each nurse to have at least 2 days off in 7 days. There are two means to model this constraint, one could use the slack variables, and another could use the inequalities feature provided by the V3 api

In [10]:
class WeekdayLeaveConstraint(ConstraintFunction):
    
    def __init__(self, X:Array, weekend):
        
        super().__init__(X)
        self._weekend = weekend
        
    def hamiltonian(self):
        row, col = self._X.shape
        
        days = col
        week_slack = Array.create("slack2", shape=(row, col), vartype="BINARY")
        print(self._X.shape)
        H = 0
        for j in range(row):
            for i in self._weekend[::2]:
                if i + 7 < days:
                    H = H + (sum(self._X[j][l] for l in range(i, i + 7)) -
                               sum(week_slack[j][l] for l in range(i, i + 5)))**2
                elif i + 5 < days and i + 7 > days:
                    H = H + (sum(self._X[j][l] for l in range(i, days)) -
                               sum(week_slack[j][l] for l in range(i, i + 5)))**2
        return H
    
    def evaluate(self, table):
        content = table.values
        rows, cols = content.shape
        failed = 0
        
        days = cols
        for r in range(rows):
            for i in self._weekend[::2]:
                if i + 7 < days:
                    if sum(content[r][i:i+7]) > 5:
                        failed += 1
                if i + 5 < days and i + 7 > days:
                    if sum(content[r][i:]) > 5:
                        failed += 1
        return 1 - (failed / (len(self._weekend[::2])*rows))

#### Weekday Leave Constraint Inequalities

The following constraint inherits the constraint defined above and share the same correctness method. The predominant difference lies on the way to model the problem. The parent class uses slack varaibles, and the child class uses inequalities.

In [11]:
class WeekdayLeaveConstraintInequalities(WeekdayLeaveConstraint):
    
    def __init__(self, X:Array, weekend):
        super().__init__(X, weekend)
        
    def inequalities(self, variables):
        row, col = self._X.shape
        hamiltonians = []
        for j in range(row):
            for i in self._weekend[::2]:
                if i + 7 < days:
                    hamiltonians.append(sum([self._X[j][l] for l in range(i, i + 7)], start=-Num(5)))
                elif i + 5 < days and i + 7 > days:
                    hamiltonians.append(sum([self._X[j][l] for l in range(i, days)], start=-Num(5)))
        
        terms = []
        
        for i in range(len(hamiltonians)):
            terms.append(convert_hamiltonian_to_binary_polynomial_term(hamiltonians[i], variables))
            
        return terms

In [62]:
class Consecutive2DaysLeaves(ConstraintFunction):
    
    def __init__(self, X):
        super().__init__(X)
        
    def hamiltonian(self):
        row, col = self._X.shape
        H = Num(0)
        for i in range(row):
            for j in range(col - 1):
                H += (1 - self._X[i][j] * self._X[i][j + 1])
                
        return H
    
    
    def evaluate(self, table):
        content = table.values
        rows, cols = content.shape
        
        all_leaves = 0
        all_consecutive_2days_leave = 0
        for r in range(rows):
            row = ''.join([str(shift) for shift in content[r]])
            all_leaves += len(re.findall(r'0+', row))
            
            consecutive_days_off = re.findall(r'(00)+0*', row)
            single_day_off = re.findall(r'0+', row)
#             print("\trow : ", row, end='=>')
#             print("\t cons days off", consecutive_days_off, end='====>')
#             print("\t single day off", single_day_off)
            
            all_consecutive_2days_leave += len(consecutive_days_off)
            
        return (all_consecutive_2days_leave / all_leaves)

## Preliminary setup

In this section, each block will setup the variables and constraints

### Variables

The shift could be generated for a specific month. The initial number of variables are the number of days times number of workers.

In [63]:
number_of_nurses = 10
year = 2022
month = 9
first_date, days = calendar.monthrange(year, month)
X = Array.create("Graveyard", shape=(number_of_nurses, days), vartype="BINARY")

### Constraints and Hamiltonian

In [64]:
lmda = 0.1
exp_workers_constraint = ExpectedNumberOfWorkersInEachShift(X, 7)
Ha = Constraint(exp_workers_constraint.hamiltonian(), "expected number of workers on each shift", )

In [65]:
lmdb = 0.5
max_consecutive_working_days = 5
max_consecutive_shifts_constraint = MaxConsecutiveShifts(X, max_consecutive_working_days)
Hb = Constraint(max_consecutive_shifts_constraint.hamiltonian(), "The Limit of max consecutive working days")

(1250,)


In [66]:
lmdc = 0.5
two_consecutive_shift = TwoConsecutiveShift(X)
Hc = Constraint(two_consecutive_shift.hamiltonian(), "Two consecutive working days")

In [67]:
# a weekday leave
weekends = get_weekend_date(year, month)
lmdd = 0.5
weekday_leave_constraint_slack_var = WeekdayLeaveConstraint(X, weekends)
Hd = Constraint(weekday_leave_constraint_slack_var.hamiltonian(), "weekday leave constraint by using slack variables")

(10, 30)


In [68]:
lmde = 1
consecutive_2_day_off = Consecutive2DaysLeaves(X)
He = Constraint(consecutive_2_day_off.hamiltonian(), "Consecutive 2 days off")

In [69]:
lmdf = 1
expected_number_of_working_days = days - len(weekends)
exp_num_working_dys_constraint = ExpectedWorkingDays(X, expected_number_of_working_days)
Hf = Constraint(exp_num_working_dys_constraint.hamiltonian(), "Expected number of working days")

### Model with slack variables

In [70]:
H_slack = lmda*Ha + lmdb*Hb + lmdc*Hc + lmdd*Hd + lmde*He + lmdf*Hf
model_slack_var = H_slack.compile()
qubo, offset = model_slack_var.to_qubo()
VARIABLES = model_slack_var.variables
matrix_terms_slack = get_matrix_term(qubo, VARIABLES)
print("Number of variables : ", len(VARIABLES))

Number of variables :  1840


In [21]:
problem_body = {}
DA_Solver = {}
DA_Solver["time_limit_sec"]= 180
DA_Solver["penalty_coef"]=10000
DA_Solver["num_run"] = 16
DA_Solver["num_group"] = 16
DA_Solver['gs_level'] = 90
DA_Solver['gs_cutoff'] = 900000
problem_body["fujitsuDA3"]=DA_Solver
problem_body["binary_polynomial"] = {'terms' : matrix_terms_slack}
job_id = post_solve(problem_body).json()['job_id']
print(job_id)

{'message': 'success'}
0f03efef-ac2a-4eb4-a69c-03997d20cbb9-231774816876673


### Evaluation

In [22]:
solutions = get_solution(job_id)

In [23]:
table = decode(solutions, 0, model_slack_var, number_of_nurses, days)
print("Expected Number of working days, average failed rates : (%.3f, %.3f)" % exp_num_working_dys_constraint.evaluate(table), end="\n######")
print("max consecutive shift : ", max_consecutive_shifts_constraint.evaluate(table), end="\n######\n")
print("consecutive 2 days-off: ", consecutive_2_day_off.evaluate(table), end="\n######\n")
print("weekday leaves: ", weekday_leave_constraint_slack_var.evaluate(table), end="\n######\n")
print("expected # of workers", exp_workers_constraint.evaluate(table), end="\n######\n")
print("at least 2 consecutive shift: ", two_consecutive_shift.evaluate(table), end="\n######\n")

[
    {
        "energy": -5260.200000000117,
        "time": 60.925
    },
    {
        "energy": -5260.800000000117,
        "time": 120.045
    },
    {
        "energy": -5261.300000000112,
        "time": 149.647
    },
    {
        "energy": -5261.600000000114,
        "time": 149.649
    },
    {
        "energy": -5261.700000000119,
        "time": 180.017
    }
]
{
    "Expected number of working days": [
        true,
        0.0
    ],
    "Consecutive 2 days off": [
        false,
        111.0
    ],
    "The Limit of max consecutive working days": [
        false,
        35.0
    ],
    "Two consecutive working days": [
        true,
        0.0
    ],
    "weekday leave constraint by using slack variables": [
        false,
        10.0
    ],
    "expected number of workers on each shift": [
        false,
        38.0
    ]
}
Expected Number of working days, average failed rates : (0.000, 0.000)
######Evaluate on the limit of max consecutive shifts:
	worker[1], date

### Replace slack variables with inequalities

In [72]:
H = lmda*Ha + lmdc*Hc + lmde*He + lmdf*Hf
model = H.compile()
qubo, offset = model.to_qubo()
VARIABLES = model.variables
matrix_terms = get_matrix_term(qubo, VARIABLES)
print("Number of variables : ", len(VARIABLES))

Number of variables :  440


#### Add inequalities

In [73]:
max_consecutive_shift_inequalities = MaxConsecutiveShiftsInequalities(X, max_consecutive_working_days)
max_consecutive_ineq_terms = max_consecutive_shift_inequalities.inequalities(VARIABLES)

weekday_leaves_ineqs = WeekdayLeaveConstraintInequalities(X, weekends)
weekday_leaves_ineq_terms = weekday_leaves_ineqs.inequalities(VARIABLES)

inequalities = []
for term in max_consecutive_ineq_terms:
    inequalities.append({
        "terms" : term
    })
    
for term in weekday_leaves_ineq_terms:
    inequalities.append({
        "terms" : term
    })

In [74]:
problem_body = {}
DA_Solver = {}
DA_Solver["time_limit_sec"]= 180
DA_Solver["penalty_coef"]=10000
DA_Solver["num_run"] = 16
DA_Solver["num_group"] = 16
DA_Solver['gs_level'] = 90
DA_Solver['gs_cutoff'] = 900000
problem_body["fujitsuDA3"]=DA_Solver
problem_body["binary_polynomial"] = { 'terms' : matrix_terms }
problem_body["inequalities"] = inequalities

In [75]:
job_id = post_solve(problem_body).json()['job_id']
print(job_id)

{'message': 'success'}
0f03efef-ac2a-4eb4-a69c-03997d20cbb9-231770835039423


### Evaluation

In [78]:
solutions = get_solution(job_id)

In [79]:
table = decode(solutions, 0, model, number_of_nurses, days)
print("Expected Number of working days, average failed rates : (%.3f, %.3f)" % exp_num_working_dys_constraint.evaluate(table), end="\n######")
print("max consecutive shift : ", max_consecutive_shifts_constraint.evaluate(table), end="\n######\n")
print("consecutive 2 days-off: ", consecutive_2_day_off.evaluate(table), end="\n######\n")
print("weekday leaves: ", weekday_leave_constraint_slack_var.evaluate(table), end="\n######\n")
print("expected # of workers", exp_workers_constraint.evaluate(table), end="\n######\n")
print("at least 2 consecutive shift: ", two_consecutive_shift.evaluate(table), end="\n######\n")

[
    {
        "energy": -5149.400000000093,
        "penalty_energy": 0,
        "time": 15.087
    },
    {
        "energy": -5149.400000000099,
        "penalty_energy": 0,
        "time": 15.088
    },
    {
        "energy": -5149.600000000102,
        "penalty_energy": 0,
        "time": 15.088
    },
    {
        "energy": -5149.800000000097,
        "penalty_energy": 0,
        "time": 22.575
    },
    {
        "energy": -5150.000000000094,
        "penalty_energy": 0,
        "time": 22.575
    },
    {
        "energy": -5150.00000000009,
        "penalty_energy": 0,
        "time": 22.576
    },
    {
        "energy": -5150.20000000009,
        "penalty_energy": 0,
        "time": 30.073
    },
    {
        "energy": -5150.600000000095,
        "penalty_energy": 0,
        "time": 30.073
    },
    {
        "energy": -5150.800000000098,
        "penalty_energy": 0,
        "time": 30.073
    },
    {
        "energy": -5151.200000000093,
        "penalty_energy": 0,
